In [21]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import time

In [81]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 128
n_epochs = 10

In [82]:
# Step 1: Read in data
# using TF Learn's built in function to load MNIST data to the folder data/mnist
mnist = input_data.read_data_sets('../data/mnist', one_hot=True) 

Extracting ../data/mnist/train-images-idx3-ubyte.gz
Extracting ../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../data/mnist/t10k-labels-idx1-ubyte.gz


In [83]:


# Step 2: create placeholders for features and labels
# each image in the MNIST data is of shape 28*28 = 784
# therefore, each image is represented with a 1x784 tensor
# there are 10 classes for each image, corresponding to digits 0 - 9. 

X = tf.placeholder(tf.float32, [batch_size, 784], name="image")
Y = tf.placeholder(tf.float32, [batch_size, 10], name="label")

# Step 3: create weights and bias
# weights and biases are initialized to 0
# shape of w depends on the dimension of X and Y so that Y = X * w + b
# shape of b depends on Y
# tf.Variable(initial_value=None, trainable=True, collections=None,
# name=None, dtype=None, ...)

# X [1,128] -> Y [1,128]

#  n * 50  50 x 128 


W = tf.Variable(tf.zeros([784, 10]))

b = tf.Variable(tf.zeros([batch_size, 10]))

# Step 4: build model
# the model that returns the logits.
# this logits will be later passed through softmax layer
# to get the probability distribution of possible label of the image
# DO NOT DO SOFTMAX HERE
# n * 784 * 784 * 10 = n * 10 

logits = tf.matmul(X, W) + b

# Step 5: define loss function
# use cross entropy loss of the real labels with the softmax of logits
# use the method:
# tf.nn.softmax_cross_entropy_with_logits(logits, Y)
# then use tf.reduce_mean to get the mean loss of the batch

entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y)

# computes the mean over all the examples in the batch
loss = tf.reduce_mean(entropy)

# Step 6: define training op
# using gradient descent to minimize loss
optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001).minimize(loss);

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(mnist.train.num_examples/batch_size)
    for i in range(n_epochs): # train the model n_epochs times
        total_loss = 0

        for _ in range(n_batches):
            X_batch, Y_batch = mnist.train.next_batch(batch_size)
            # TO-DO: run optimizer + fetch loss_batch
            _, loss_batch = sess.run([optimizer, loss], feed_dict={X:X_batch, Y:Y_batch})
            # 
            total_loss += loss_batch
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    
    print(sess.run([W, b]))
    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!') # should be around 0.35 after 25 epochs

    # test the model
    n_batches = int(mnist.test.num_examples/batch_size)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(batch_size)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], feed_dict={X: X_batch, Y:Y_batch}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
        total_correct_preds += sess.run(accuracy)

    print('Accuracy {0}'.format(total_correct_preds/mnist.test.num_examples))

Average loss epoch 0: 0.9007256313950032
Average loss epoch 1: 0.3537150853094243
Average loss epoch 2: 0.31420814251149454
Average loss epoch 3: 0.2979091649358367
Average loss epoch 4: 0.28906390972279167
Average loss epoch 5: 0.28235641691512436
Average loss epoch 6: 0.2777258518449354
Average loss epoch 7: 0.2751313379872373
Average loss epoch 8: 0.27225675388719095
Average loss epoch 9: 0.270220017481795
[array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), array([[-0.21514976,  0.19550693, -0.20331845, ..., -0.2052836 ,
        -0.40582335, -0.16733116],
       [-0.14972785,  0.08639303, -0.30367988, ..., -0.10951662,
        -0.36489853, -0.16943038],
       [-0.17570059,  0.14575151, -0.32022366, ..., -0.11363158,
        -0.43118909, -0.208

## 不同优化方法的准确率
* GradientDescentOptimizer:0.8429
* AdadeltaOptimizer:0.7062
* AdagradOptimizer:0.851
* AdamOptimizer:0.924
* FtrlOptimizer:0.8507
* RMSPropOptimizer:0.9249

## RMSPropOptimizer 不同学习率优化结果
* 0.001: 0.9241
* 0.005: 0.9258
* 0.01:  0.9249


## RMSPropOptimizer 不同迭代次数的结果
learning_rate = 0.01

* 10次：0.9241
* 30次：0.9255
* 100次：0.926